#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter Monthly Budget Mover Parameters
Apply the previous month's budget/spend delta to the current month.  Aggregate up the budget and spend from the previous month of each category declared then apply the delta of the spend and budget equally to each Line Item under that Category.
 1. No changes made can be made in DV360 from the start to the end of this process
 1. Make sure there is budget information for the current and previous month's IOs in DV360
 1. Make sure the provided spend report has spend data for every IO in the previous month
 1. Spend report must contain 'Revenue (Adv Currency)' and 'Insertion Order ID'
 1. There are no duplicate IO Ids in the categories outlined below
 1. This process must be ran during the month of the budget it is updating
 1. If you receive a 502 error then you must separate your jobs into two, because there is too much information being pulled in the sdf
 1. Manually run this job
 1. Once the job has completed go to the table for the new sdf and export to a csv
 1. Take the new sdf and upload it into DV360
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'recipe_name': '',  # 
  'auth_write': 'service',  # Credentials used for writing data.
  'auth_read': 'user',  # Credentials used for reading data.
  'partner_id': '',  # The sdf file types.
  'budget_categories': '{}',  # A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}
  'filter_ids': [],  # Comma separated list of filter ids for the request.
  'excluded_ios': '',  # A comma separated list of Inserion Order Ids that should be exluded from the budget calculations
  'version': '5',  # The sdf version to be returned.
  'is_colab': True,  # Are you running this in Colab? (This will store the files in Colab instead of Bigquery)
  'dataset': '',  # Dataset that you would like your output tables to be produced in.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute Monthly Budget Mover
This does NOT need to be modified unles you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'description': 'Create a dataset where data will be combined and transfored for upload.',
      'auth': 'user',
      'dataset': {'field': {'name': 'dataset','kind': 'string','order': 1,'description': 'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'timeout': 90,
        'filters': {
          'FILTER_ADVERTISER': {
            'values': {'field': {'name': 'filter_ids','kind': 'integer_list','order': 7,'default': '','description': 'The comma separated list of Advertiser Ids.'}}
          }
        },
        'body': {
          'timezoneCode': {'field': {'name': 'recipe_timezone','kind': 'timezone','description': 'Timezone for report dates.','default': 'America/Los_Angeles'}},
          'metadata': {
            'title': {'field': {'name': 'recipe_name','kind': 'string','prefix': 'Monthly_Budget_Mover_','order': 1,'description': 'Name of report in DV360, should be unique.'}},
            'dataRange': 'PREVIOUS_MONTH',
            'format': 'CSV'
          },
          'params': {
            'type': 'TYPE_GENERAL',
            'groupBys': [
              'FILTER_ADVERTISER_CURRENCY',
              'FILTER_INSERTION_ORDER'
            ],
            'metrics': [
              'METRIC_REVENUE_ADVERTISER'
            ]
          }
        }
      },
      'delete': False
    }
  },
  {
    'monthly_budget_mover': {
      'auth': 'user',
      'is_colab': {'field': {'name': 'is_colab','kind': 'boolean','default': True,'order': 7,'description': 'Are you running this in Colab? (This will store the files in Colab instead of Bigquery)'}},
      'report_name': {'field': {'name': 'recipe_name','kind': 'string','prefix': 'Monthly_Budget_Mover_','order': 1,'description': 'Name of report in DV360, should be unique.'}},
      'budget_categories': {'field': {'name': 'budget_categories','kind': 'json','order': 3,'default': '{}','description': 'A dictionary to show which IO Ids go under which Category. {"CATEGORY1":[12345,12345,12345], "CATEGORY2":[12345,12345]}'}},
      'excluded_ios': {'field': {'name': 'excluded_ios','kind': 'integer_list','order': 4,'description': 'A comma separated list of Inserion Order Ids that should be exluded from the budget calculations'}},
      'sdf': {
        'auth': 'user',
        'version': {'field': {'name': 'version','kind': 'choice','order': 6,'default': '5','description': 'The sdf version to be returned.','choices': ['SDF_VERSION_5','SDF_VERSION_5_1']}},
        'partner_id': {'field': {'name': 'partner_id','kind': 'integer','order': 1,'description': 'The sdf file types.'}},
        'file_types': 'INSERTION_ORDER',
        'filter_type': 'FILTER_TYPE_ADVERTISER_ID',
        'read': {
          'filter_ids': {
            'single_cell': True,
            'values': {'field': {'name': 'filter_ids','kind': 'integer_list','order': 4,'default': [],'description': 'Comma separated list of filter ids for the request.'}}
          }
        },
        'time_partitioned_table': False,
        'create_single_day_table': False,
        'dataset': {'field': {'name': 'dataset','kind': 'string','order': 6,'default': '','description': 'Dataset to be written to in BigQuery.'}},
        'table_suffix': ''
      },
      'out_old_sdf': {
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 8,'default': '','description': 'Dataset that you would like your output tables to be produced in.'}},
          'table': {'field': {'name': 'recipe_name','kind': 'string','prefix': 'SDF_OLD_','description': ''}},
          'schema': [
          ],
          'skip_rows': 0,
          'disposition': 'WRITE_TRUNCATE'
        },
        'file': '/content/old_sdf.csv'
      },
      'out_new_sdf': {
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 8,'default': '','description': 'Dataset that you would like your output tables to be produced in.'}},
          'table': {'field': {'name': 'recipe_name','kind': 'string','prefix': 'SDF_NEW_','description': ''}},
          'schema': [
          ],
          'skip_rows': 0,
          'disposition': 'WRITE_TRUNCATE'
        },
        'file': '/content/new_sdf.csv'
      },
      'out_changes': {
        'bigquery': {
          'dataset': {'field': {'name': 'dataset','kind': 'string','order': 8,'default': '','description': 'Dataset that you would like your output tables to be produced in.'}},
          'table': {'field': {'name': 'recipe_name','kind': 'string','prefix': 'SDF_BUDGET_MOVER_LOG_','description': ''}},
          'schema': [
          ],
          'skip_rows': 0,
          'disposition': 'WRITE_TRUNCATE'
        },
        'file': '/content/log.csv'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
